In [1]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 75.7 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.76
    Uninstalling botocore-1.29.76:
      Successfully uninstalled botocore-1.29.76
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.29.154 which is incompatible.


In [2]:
!cp -r /kaggle/input/datasetuno/cable/* /kaggle/working

In [1]:
import boto3
import uuid
import os
import torch
import torch.utils.data as data
from PIL import Image
from scipy.stats import ttest_ind
from sklearn.metrics import roc_auc_score
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import transforms

ACCESS_KEY = os.environ.get('AWS_ACCESS_KEY_ID')
SECRET_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
BUCKET_NAME = os.environ.get('BUCKET_NAME')

DATASET_NAME = "cable"
GENERATOR_NAME = "generator_weights.pt"
GENERATOR_PATH = f"{DATASET_NAME}/{GENERATOR_NAME}"
DESCRIMINATOR_NAME = "descriminator_weights.pt"
DESCRIMINATOR_PATH = f"{DATASET_NAME}/{DESCRIMINATOR_NAME}"
IMAGES_NAME = "images.zip"
session = boto3.Session(
    aws_access_key_id=ACCESS_KEY, 
    aws_secret_access_key=SECRET_KEY
)

s3 = session.resource('s3')
bucket = s3.Bucket(BUCKET_NAME)

ModuleNotFoundError: No module named 'boto3'

## DOWNLOADING THE GENERATOR

In [4]:
try:
    bucket.download_file(Key=GENERATOR_PATH, Filename=GENERATOR_NAME)
except:
    print("The generator weights don't exist.")

## DOWNLOADING THE DESCRIMINATOR

In [5]:
try:
    bucket.download_file(Key=DESCRIMINATOR_PATH, Filename=DESCRIMINATOR_NAME)
except:
    print("The descriminator weights don't exist.")

## DCGAN Architecture

In [6]:
!python dcgan.py --n_epochs 100 --img_size 256 --channels 3 --sample_interval 50 --train_data True --augment_data True --load_weights True --save_path "augment" --count 200 --weights_filename "weights.pt"

Namespace(n_epochs=100, batch_size=64, lr=0.0002, b1=0.5, b2=0.999, n_cpu=8, latent_dim=100, img_size=256, channels=3, sample_interval=50, train_data=True, save_weights=False, load_weights=True, augment_data=True, save_path='augment', count=200, weights_filename='weights.pt')
Loaded weights from generator_weights.pt (epoch 20000)
Loaded weights from descriminator_weights.pt (epoch 20000)
[Epoch 0/100] [Batch 0/1] [D loss: 0.310877] [G loss: 2.859544]
[Epoch 1/100] [Batch 0/1] [D loss: 0.117045] [G loss: 4.337185]
[Epoch 2/100] [Batch 0/1] [D loss: 0.143007] [G loss: 4.712182]
[Epoch 3/100] [Batch 0/1] [D loss: 0.080592] [G loss: 6.055743]
[Epoch 4/100] [Batch 0/1] [D loss: 0.119871] [G loss: 4.950448]
[Epoch 5/100] [Batch 0/1] [D loss: 0.115417] [G loss: 3.749254]
[Epoch 6/100] [Batch 0/1] [D loss: 0.076113] [G loss: 4.540829]
[Epoch 7/100] [Batch 0/1] [D loss: 0.221302] [G loss: 5.162452]
[Epoch 8/100] [Batch 0/1] [D loss: 0.113531] [G loss: 5.782274]
[Epoch 9/100] [Batch 0/1] [D loss

In [7]:
!zip -r augment.zip augment/*

  adding: augment/generated_image_0.jpg (deflated 1%)
  adding: augment/generated_image_1.jpg (deflated 1%)
  adding: augment/generated_image_10.jpg (deflated 1%)
  adding: augment/generated_image_100.jpg (deflated 1%)
  adding: augment/generated_image_101.jpg (deflated 1%)
  adding: augment/generated_image_102.jpg (deflated 1%)
  adding: augment/generated_image_103.jpg (deflated 1%)
  adding: augment/generated_image_104.jpg (deflated 1%)
  adding: augment/generated_image_105.jpg (deflated 1%)
  adding: augment/generated_image_106.jpg (deflated 1%)
  adding: augment/generated_image_107.jpg (deflated 1%)
  adding: augment/generated_image_108.jpg (deflated 0%)
  adding: augment/generated_image_109.jpg (deflated 1%)
  adding: augment/generated_image_11.jpg (deflated 1%)
  adding: augment/generated_image_110.jpg (deflated 1%)
  adding: augment/generated_image_111.jpg (deflated 1%)
  adding: augment/generated_image_112.jpg (deflated 1%)
  adding: augment/generated_image_113.jpg (deflated 1%

## Evaluate the DCGAN Augmented images

In [8]:
!cp -r /kaggle/input/cable/* /kaggle/working

In [9]:
!cp -r /kaggle/working/augment/* /kaggle/working/cable/train/good

In [10]:
!cp -r /kaggle/working/augment/* /kaggle/working/cable/test/good

In [11]:
data_dir = 'cable'
train_dir = os.path.join(data_dir,'train')
test_dir = os.path.join(data_dir,'test')

In [12]:
class ImageDataset(data.Dataset):
    def __init__(self, data_dir, transform=None):
        super(ImageDataset, self).__init__()
        self.data_dir = data_dir
        self.transform = transform
        self.data = []
        self.targets = []

        # Iterate over the data directory and load images
        for root, dirs, files in os.walk(data_dir):
            for file in files:
                image_path = os.path.join(root, file)
                image = Image.open(image_path)
                self.data.append(image)

                # Determine the class label based on the directory name
                class_label = os.path.basename(root)
                if class_label == "good":
                    self.targets.append(1)  # Assign class 1 for "good"
                elif class_label == "bad":
                    self.targets.append(0)  # Assign class 0 for "bad"

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data[index]
        target = self.targets[index]

        if self.transform:
            image = self.transform(image)

        return image, target

In [2]:
import os
import torch
import torch.utils.data as data
from PIL import Image
from scipy.stats import ttest_ind
from sklearn.metrics import roc_auc_score
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import transforms

ModuleNotFoundError: No module named 'torchvision'

In [13]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(32 * 16 * 16, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
from torchsummary import summary
model = CNN()
summary(model, (3, 256, 256))

In [14]:
transform = transforms.Compose([
    transforms.RandomRotation(50),  # Rotate randomly between -50 and 50 degrees
    transforms.RandomResizedCrop(256, scale=(0.8, 1.0)),  # Random scale and crop to 256x256
    transforms.RandomHorizontalFlip(),  # Flip horizontally with a 50% chance
    transforms.ToTensor(),
])

# Create data loaders
train_dataset = ImageDataset(train_dir, transform=transform)
test_dataset = ImageDataset(test_dir, transform=transform)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Set device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the CNN model
model = CNN().to(device)


# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=0.002)

In [15]:
# Training loop
num_epochs = 500
model.train()

for epoch in range(num_epochs):
    for images, targets in train_loader:
        # Move data to device
        images = images.to(device)
        targets = targets.unsqueeze(1).to(device)
        
        # Convert targets to float
        targets = targets.float()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training loss
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/500], Loss: 0.6543
Epoch [1/500], Loss: 0.2506
Epoch [1/500], Loss: 0.0001
Epoch [1/500], Loss: 6.2500
Epoch [1/500], Loss: 0.0000
Epoch [1/500], Loss: 3.1250
Epoch [1/500], Loss: 0.0000
Epoch [1/500], Loss: 6.2500
Epoch [1/500], Loss: 0.0000
Epoch [1/500], Loss: 3.1250
Epoch [1/500], Loss: 0.0000
Epoch [1/500], Loss: 6.2500
Epoch [1/500], Loss: 3.1250
Epoch [1/500], Loss: 0.0000
Epoch [2/500], Loss: 6.2500
Epoch [2/500], Loss: 6.2500
Epoch [2/500], Loss: 0.0000
Epoch [2/500], Loss: 3.1250
Epoch [2/500], Loss: 0.0000
Epoch [2/500], Loss: 0.0000
Epoch [2/500], Loss: 3.1250
Epoch [2/500], Loss: 3.1250
Epoch [2/500], Loss: 3.1250
Epoch [2/500], Loss: 0.0000
Epoch [2/500], Loss: 0.0000
Epoch [2/500], Loss: 3.1250
Epoch [2/500], Loss: 3.1250
Epoch [2/500], Loss: 0.0000
Epoch [3/500], Loss: 3.1250
Epoch [3/500], Loss: 3.1250
Epoch [3/500], Loss: 0.0000
Epoch [3/500], Loss: 3.1250
Epoch [3/500], Loss: 3.1250
Epoch [3/500], Loss: 3.1250
Epoch [3/500], Loss: 0.0000
Epoch [3/500], Loss:

In [16]:
# Evaluation
model.eval()
with torch.no_grad():
    all_predictions = []
    all_targets = []
    for images, targets in test_loader:
        
        # Move data to device
        images = images.to(device)
        targets = targets.unsqueeze(1).to(device)

        # Forward pass
        outputs = model(images)
        predictions = torch.round(outputs)

        # Collect predictions and targets
        all_predictions.extend(predictions.cpu().numpy().flatten().tolist())
        all_targets.extend(targets.cpu().numpy().flatten().tolist())

    # Calculate metrics
    correct = sum(all_predictions[i] == all_targets[i] for i in range(len(all_predictions)))
    accuracy = correct / len(all_predictions)
    tp = sum(all_predictions[i] == 1 and all_targets[i] == 1 for i in range(len(all_predictions)))
    tn = sum(all_predictions[i] == 0 and all_targets[i] == 0 for i in range(len(all_predictions)))
    fp = sum(all_predictions[i] == 1 and all_targets[i] == 0 for i in range(len(all_predictions)))
    fn = sum(all_predictions[i] == 0 and all_targets[i] == 1 for i in range(len(all_predictions)))
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    auc = roc_auc_score(all_targets, all_predictions)
    p_value = ttest_ind(all_predictions, all_targets).pvalue

/tmp/ipykernel_73/1521511933.py:30: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  p_value = ttest_ind(all_predictions, all_targets).pvalue


In [17]:
# Print the evaluation results
print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("AUC:", auc)
print("p-value:", p_value)

Accuracy: 0.7588235294117647
Sensitivity: 1.0
Specificity: 0.0
AUC: 0.5
p-value: 1.5857181125501287e-23
